In [ ]:
#Churn rate
SELECT 100. / 2000;

SELECT 450. / 3000;

In [ ]:
#Single Month I
SELECT 1.0 * 
(
  SELECT COUNT(*)
  FROM subscriptions
  WHERE subscription_start < '2017-01-01'
  AND (
    subscription_end
    BETWEEN '2017-01-01'
    AND '2017-01-31'
  )
) / (
  SELECT COUNT(*) 
  FROM subscriptions 
  WHERE subscription_start < '2017-01-01'
  AND (
    (subscription_end >= '2017-01-01')
    OR (subscription_end IS NULL)
  )
) 
AS result;

In [ ]:
#Single Month II
WITH status AS 
(SELECT
CASE
  WHEN (subscription_end BETWEEN '2017-01-01'
        AND '2017-01-31') THEN 1
    ELSE 0
  END AS is_canceled,
CASE 
  WHEN subscription_start < '2017-01-01'
    AND (
      (subscription_end >= '2017-01-01')
      OR (subscription_end IS NULL)
    ) THEN 1
   ELSE 0
END AS is_active
FROM subscriptions
)
  
SELECT 1.0 * SUM(is_canceled) / SUM(is_active)
FROM status;

In [ ]:
#Multiple Months
WITH months AS (
  SELECT 
  '2017-01-01' AS first_day,
  '2017-01-31' AS last_day
  UNION
  SELECT 
   '2017-02-01' AS first_day,
  '2017-02-28' AS last_day
  UNION
  SELECT 
  '2017-03-01' AS first_day,
  '2017-03-31' AS last_day
)
SELECT *
FROM months;

In [ ]:
#Multiple Months II
WITH months AS
(SELECT
  '2017-01-01' as first_day,
  '2017-01-31' as last_day
UNION
SELECT
  '2017-02-01' as first_day,
  '2017-02-28' as last_day
UNION
SELECT
  '2017-03-01' as first_day,
  '2017-03-31' as last_day
),
cross_join AS (
SELECT *
FROM subscriptions
CROSS JOIN months)
SELECT *
FROM cross_join
LIMIT 100;

In [ ]:
#Multiple Months III
WITH months AS
(SELECT
'2017-01-01' as first_day,
'2017-01-31' as last_day
UNION
SELECT
'2017-02-01' as first_day,
'2017-02-28' as last_day
UNION
SELECT
'2017-03-01' as first_day,
'2017-03-31' as last_day
),
cross_join AS
(SELECT *
FROM subscriptions
CROSS JOIN months),
status AS
(SELECT id, first_day AS month,
CASE
WHEN (subscription_start < first_day)
AND (
subscription_end > first_day
OR subscription_end IS NULL
) THEN 1
ELSE 0
END AS 'is_active'
FROM cross_join)
SELECT *
FROM status
LIMIT 100;

In [ ]:
#Multiple Month IV
WITH months AS
(SELECT
  '2017-01-01' as first_day,
  '2017-01-31' as last_day
UNION
SELECT
  '2017-02-01' as first_day,
  '2017-02-28' as last_day
UNION
SELECT
  '2017-03-01' as first_day,
  '2017-03-31' as last_day
),
cross_join AS
(SELECT *
FROM subscriptions
CROSS JOIN months),
status AS
(SELECT id, first_day as month,
CASE
  WHEN (subscription_start < first_day)
    AND (
      subscription_end > first_day
      OR subscription_end IS NULL
    ) THEN 1
  ELSE 0
END as is_active,
CASE
WHEN subscription_end BETWEEN first_day AND last_day
THEN 1
ELSE 0
END AS 'is_canceled'
FROM cross_join)
SELECT *
FROM status
LIMIT 100;

In [ ]:
#Multiple Month V
WITH months AS
(SELECT
  '2017-01-01' as first_day,
  '2017-01-31' as last_day
UNION
SELECT
  '2017-02-01' as first_day,
  '2017-02-28' as last_day
UNION
SELECT
  '2017-03-01' as first_day,
  '2017-03-31' as last_day
),
cross_join AS
(SELECT *
FROM subscriptions
CROSS JOIN months),
status AS
(SELECT id, first_day as month,
CASE
  WHEN (subscription_start < first_day)
    AND (
      subscription_end > first_day
      OR subscription_end IS NULL
    ) THEN 1
  ELSE 0
END as is_active,
CASE 
  WHEN subscription_end BETWEEN first_day AND last_day THEN 1
  ELSE 0
END as is_canceled
FROM cross_join),
status_aggregate AS (
SELECT month, 
SUM(is_active) AS active,
SUM(is_canceled) AS cancelled
FROM status
GROUP BY month)
SELECT *
FROM status_aggregate;

In [ ]:
#Multiple Month VI
WITH months AS
(SELECT
  '2017-01-01' as first_day,
  '2017-01-31' as last_day
UNION
SELECT
  '2017-02-01' as first_day,
  '2017-02-28' as last_day
UNION
SELECT
  '2017-03-01' as first_day,
  '2017-03-31' as last_day
),
cross_join AS
(SELECT *
FROM subscriptions
CROSS JOIN months),
status AS
(SELECT id, first_day as month,
CASE
  WHEN (subscription_start < first_day)
    AND (
      subscription_end > first_day
      OR subscription_end IS NULL
    ) THEN 1
  ELSE 0
END as is_active,
CASE 
  WHEN subscription_end BETWEEN first_day AND last_day THEN 1
  ELSE 0
END as is_canceled
FROM cross_join),
status_aggregate AS
(SELECT
  month,
  SUM(is_active) as active,
  SUM(is_canceled) as canceled
FROM status
GROUP BY month)
SELECT month, 
1.0 * status_aggregate.canceled / status_aggregate.active AS churn_rate
FROM status_aggregate;